<a href="https://colab.research.google.com/github/gmflau/GenAI/blob/main/colab/The_Crispy_mixedbread_Rerank_Models_using_Remote_Ollama_powered_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### This Colab demonstrates how Mixedbread Reranker improves relevancy in LLM responses *without* a vector database.



In [1]:
!pip install -U -q sentence-transformers langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.0 MB/s eta 0:00:00


Here is a quick example: Given the query “Who wrote 'To Kill a Mockingbird'?”, we want to retrieve the most relevant passage to that query.

In [2]:
from sentence_transformers import CrossEncoder
from langchain_core.documents import Document

# Load the model, here we use our base sized model
model = CrossEncoder("mixedbread-ai/mxbai-rerank-base-v1")


# Example query and documents
query = "Who wrote 'To Kill a Mockingbird'?"
documents = [
"'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature.",
"The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil.",
"Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961.",
"Jane Austen was an English novelist known primarily for her six major novels, which interpret, critique and comment upon the British landed gentry at the end of the 18th century.",
"The 'Harry Potter' series, which consists of seven fantasy novels written by British author J.K. Rowling, is among the most popular and critically acclaimed books of the modern era.",
"'The Great Gatsby', a novel written by American author F. Scott Fitzgerald, was published in 1925. The story is set in the Jazz Age and follows the life of millionaire Jay Gatsby and his pursuit of Daisy Buchanan."
]

# Calculate the scores
results = model.rank(query, documents, return_documents=True, top_k=3)

documents=[Document(d['text']) for d in results]
documents

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

[Document(page_content="'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature."),
 Document(page_content="Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961."),
 Document(page_content="The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil.")]

In [7]:
from langchain.llms import Ollama

ollama = Ollama(base_url='https://aa30-34-83-152-22.ngrok-free.app', model="gemma:2b")
#ollama = Ollama(base_url='', model="llama2")
#ollama = Ollama(base_url='', model="mistral")


In [6]:
%%time

from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "Who wrote 'To Kill a Mockingbird'?:\n\n{context}")]
)

chain = create_stuff_documents_chain(ollama, prompt)

response = chain.invoke({"context": documents})
print(response)

The author of 'To Kill a Mockingbird' is Harper Lee.

According to the passage, Harper Lee was born in Monroeville, Alabama in 1926 and received the Pulitzer Prize for Fiction in 1961 for her novel 'To Kill a Mockingbird'.
CPU times: user 601 ms, sys: 80.2 ms, total: 681 ms
Wall time: 1min 20s
